## Day 82 Lecture 1 Assignment

In this assignment, we will learn about activation functions. We will create a neural network and measure the model's performance using different activations.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


We will import the famous titanic dataset below and produce a neural network that will predict the chance of survival for a passenger.

In [2]:
titanic = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/titanic.csv')

In [3]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We'll perform some feature engineering

Let's start by keeping only the columns we'd like to use for our analysis. Keep only the columns: Survived, Pclass, Sex, SibSp, Parch, and Embarked

In [4]:
# Answer below
an_cols = ['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
model_df = titanic[an_cols]
model_df

,Survived,Pclass,Sex,SibSp,Parch,Embarked
0,0,3,male,1,0,S
1,1,1,female,1,0,C
2,1,3,female,0,0,S
3,1,1,female,1,0,S
4,0,3,male,0,0,S
...,...,...,...,...,...,...
886,0,2,male,0,0,S
887,1,1,female,0,0,S
888,0,3,female,1,2,S
889,1,1,male,0,0,C


Now examine how many rows contain missing data. Given how much missing data we have, should we remove the column with the most missing data, or remove all rows containing missing data? Do what you think is best.

In [5]:
# Answer below: 
model_df.isna().mean()

Survived    0.000000
Pclass      0.000000
Sex         0.000000
SibSp       0.000000
Parch       0.000000
Embarked    0.002245
dtype: float64

In [6]:
model_df = model_df.dropna()
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Survived  889 non-null    int64 
 1   Pclass    889 non-null    int64 
 2   Sex       889 non-null    object
 3   SibSp     889 non-null    int64 
 4   Parch     889 non-null    int64 
 5   Embarked  889 non-null    object
dtypes: int64(4), object(2)
memory usage: 48.6+ KB


Now we'll create a one hot encoding of the variables Pclass, sex, and Embarked

In [7]:
# Answer below:
dummy_df = pd.get_dummies(model_df, columns=['Pclass', 'Sex', 'Embarked'], drop_first=True)
dummy_df

,Survived,SibSp,Parch,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,0,1,0,0,1,1,0,1
1,1,1,0,0,0,0,0,0
2,1,0,0,0,1,0,0,1
3,1,1,0,0,0,0,0,1
4,0,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...
886,0,0,0,1,0,1,0,1
887,1,0,0,0,0,0,0,1
888,0,1,2,0,1,0,0,1
889,1,0,0,0,0,1,0,0


Split the data into train and test. 20% of the data should be set aside for testing. Use Survived as your target variable.

In [8]:
# Answer below
X = dummy_df.drop(columns=['Survived'])
y = dummy_df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=66)

At this point, we are ready to create a model. Import `Sequential` and `Dense` from Keras

In [9]:
# Answer below:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Create a model with 5 layers. The first layer should be a dense layer that receives the input, the last layer should be of size 1. You determine the remaining layer sizes.

Use a tanh activation for the output layer.

In [10]:
# Answer below
model = Sequential()

# first dense layer
model.add(Dense(8, input_dim=X_train.shape[1], activation='relu'))

# hidden layers
model.add(Dense(6, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(6, activation='relu'))

# output dense layer
model.add(Dense(1, activation='tanh'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 64        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 7         
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


Compile the model using the adam optimizer, binary crossentropy loss, and the accuracy metric.

Fit the model using a batch size of 80 over 200 epochs.

In [12]:
# Answer below:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train, epochs=200, batch_size=80, validation_data=(X_test, y_test))

Epoch 1/200
9/9 [==============================] - 0s 17ms/step - loss: 3.8469 - accuracy: 0.6104 - val_loss: 2.1835 - val_accuracy: 0.5674
Epoch 2/200
9/9 [==============================] - 0s 3ms/step - loss: 1.6329 - accuracy: 0.5935 - val_loss: 1.4017 - val_accuracy: 0.5506
Epoch 3/200
9/9 [==============================] - 0s 3ms/step - loss: 1.0952 - accuracy: 0.5837 - val_loss: 0.9592 - val_accuracy: 0.5393
Epoch 4/200
9/9 [==============================] - 0s 2ms/step - loss: 0.9287 - accuracy: 0.5696 - val_loss: 0.9098 - val_accuracy: 0.5337
Epoch 5/200
9/9 [==============================] - 0s 3ms/step - loss: 0.8747 - accuracy: 0.5724 - val_loss: 0.8859 - val_accuracy: 0.5393
Epoch 6/200
9/9 [==============================] - 0s 3ms/step - loss: 0.8549 - accuracy: 0.5696 - val_loss: 0.8696 - val_accuracy: 0.5337
Epoch 7/200
9/9 [==============================] - 0s 3ms/step - loss: 0.8404 - accuracy: 0.5710 - val_loss: 0.8569 - val_accuracy: 0.5337
Epoch 8/200
9/9 [=========

9/9 [==============================] - 0s 3ms/step - loss: 0.5612 - accuracy: 0.6864 - val_loss: 0.5662 - val_accuracy: 0.6685
Epoch 60/200
9/9 [==============================] - 0s 3ms/step - loss: 0.5579 - accuracy: 0.6892 - val_loss: 0.5635 - val_accuracy: 0.6854
Epoch 61/200
9/9 [==============================] - 0s 2ms/step - loss: 0.5563 - accuracy: 0.6878 - val_loss: 0.5612 - val_accuracy: 0.6854
Epoch 62/200
9/9 [==============================] - 0s 2ms/step - loss: 0.5538 - accuracy: 0.7103 - val_loss: 0.5590 - val_accuracy: 0.7079
Epoch 63/200
9/9 [==============================] - 0s 2ms/step - loss: 0.5525 - accuracy: 0.7117 - val_loss: 0.5568 - val_accuracy: 0.7247
Epoch 64/200
9/9 [==============================] - 0s 2ms/step - loss: 0.5501 - accuracy: 0.7145 - val_loss: 0.5549 - val_accuracy: 0.7247
Epoch 65/200
9/9 [==============================] - 0s 2ms/step - loss: 0.5483 - accuracy: 0.7173 - val_loss: 0.5530 - val_accuracy: 0.7247
Epoch 66/200
9/9 [===============

9/9 [==============================] - 0s 2ms/step - loss: 0.5206 - accuracy: 0.7271 - val_loss: 0.5207 - val_accuracy: 0.7640
Epoch 118/200
9/9 [==============================] - 0s 2ms/step - loss: 0.5185 - accuracy: 0.7454 - val_loss: 0.5229 - val_accuracy: 0.7528
Epoch 119/200
9/9 [==============================] - 0s 2ms/step - loss: 0.5114 - accuracy: 0.7482 - val_loss: 0.5118 - val_accuracy: 0.7640
Epoch 120/200
9/9 [==============================] - 0s 2ms/step - loss: 0.5061 - accuracy: 0.7581 - val_loss: 0.5073 - val_accuracy: 0.7921
Epoch 121/200
9/9 [==============================] - 0s 3ms/step - loss: 0.5049 - accuracy: 0.7707 - val_loss: 0.5037 - val_accuracy: 0.7753
Epoch 122/200
9/9 [==============================] - 0s 3ms/step - loss: 0.5014 - accuracy: 0.7736 - val_loss: 0.5024 - val_accuracy: 0.7921
Epoch 123/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4997 - accuracy: 0.7806 - val_loss: 0.5013 - val_accuracy: 0.7978
Epoch 124/200
9/9 [========

9/9 [==============================] - 0s 3ms/step - loss: 0.4654 - accuracy: 0.8059 - val_loss: 0.4522 - val_accuracy: 0.8034
Epoch 176/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4652 - accuracy: 0.8045 - val_loss: 0.4516 - val_accuracy: 0.7865
Epoch 177/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4651 - accuracy: 0.7975 - val_loss: 0.4510 - val_accuracy: 0.7865
Epoch 178/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4649 - accuracy: 0.8045 - val_loss: 0.4508 - val_accuracy: 0.7865
Epoch 179/200
9/9 [==============================] - 0s 2ms/step - loss: 0.4645 - accuracy: 0.8003 - val_loss: 0.4501 - val_accuracy: 0.7978
Epoch 180/200
9/9 [==============================] - 0s 2ms/step - loss: 0.4636 - accuracy: 0.8003 - val_loss: 0.4499 - val_accuracy: 0.7865
Epoch 181/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4638 - accuracy: 0.8045 - val_loss: 0.4497 - val_accuracy: 0.7865
Epoch 182/200
9/9 [========

Redefine the model using a sigmoid activation for the last layer. What is the difference in accuracy.

In [15]:
# Answer below
model = Sequential()

# first dense layer
model.add(Dense(8, input_dim=X_train.shape[1], activation='relu'))

# hidden layers
model.add(Dense(6, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(6, activation='relu'))

# output dense layer
model.add(Dense(1, activation='sigmoid'))

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 8)                 64        
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


Compile the model using the adam optimizer, binary crossentropy loss, and the accuracy metric.

Fit the model using a batch size of 80 over 200 epochs.

In [17]:
# Answer below:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, epochs=200, batch_size=80, validation_data=(X_test, y_test))

Epoch 1/200
9/9 [==============================] - 0s 14ms/step - loss: 0.6893 - accuracy: 0.6273 - val_loss: 0.6887 - val_accuracy: 0.5843
Epoch 2/200
9/9 [==============================] - 0s 3ms/step - loss: 0.6851 - accuracy: 0.6259 - val_loss: 0.6857 - val_accuracy: 0.5843
Epoch 3/200
9/9 [==============================] - 0s 3ms/step - loss: 0.6802 - accuracy: 0.6259 - val_loss: 0.6825 - val_accuracy: 0.5843
Epoch 4/200
9/9 [==============================] - 0s 3ms/step - loss: 0.6750 - accuracy: 0.6259 - val_loss: 0.6788 - val_accuracy: 0.5843
Epoch 5/200
9/9 [==============================] - 0s 3ms/step - loss: 0.6699 - accuracy: 0.6259 - val_loss: 0.6748 - val_accuracy: 0.5843
Epoch 6/200
9/9 [==============================] - 0s 3ms/step - loss: 0.6637 - accuracy: 0.6259 - val_loss: 0.6705 - val_accuracy: 0.5843
Epoch 7/200
9/9 [==============================] - 0s 3ms/step - loss: 0.6572 - accuracy: 0.6259 - val_loss: 0.6661 - val_accuracy: 0.5843
Epoch 8/200
9/9 [=========

9/9 [==============================] - 0s 3ms/step - loss: 0.4427 - accuracy: 0.8045 - val_loss: 0.4092 - val_accuracy: 0.8034
Epoch 60/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4415 - accuracy: 0.8045 - val_loss: 0.4070 - val_accuracy: 0.8034
Epoch 61/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4413 - accuracy: 0.8045 - val_loss: 0.4048 - val_accuracy: 0.8034
Epoch 62/200
9/9 [==============================] - 0s 2ms/step - loss: 0.4412 - accuracy: 0.8045 - val_loss: 0.4056 - val_accuracy: 0.8034
Epoch 63/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4406 - accuracy: 0.8003 - val_loss: 0.4068 - val_accuracy: 0.8146
Epoch 64/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4404 - accuracy: 0.8073 - val_loss: 0.4041 - val_accuracy: 0.8034
Epoch 65/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4402 - accuracy: 0.8045 - val_loss: 0.4058 - val_accuracy: 0.8034
Epoch 66/200
9/9 [===============

9/9 [==============================] - 0s 3ms/step - loss: 0.4301 - accuracy: 0.8101 - val_loss: 0.3924 - val_accuracy: 0.8258
Epoch 118/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4306 - accuracy: 0.8115 - val_loss: 0.3942 - val_accuracy: 0.8202
Epoch 119/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4298 - accuracy: 0.8087 - val_loss: 0.3926 - val_accuracy: 0.8315
Epoch 120/200
9/9 [==============================] - 0s 4ms/step - loss: 0.4297 - accuracy: 0.8087 - val_loss: 0.3927 - val_accuracy: 0.8315
Epoch 121/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4294 - accuracy: 0.8101 - val_loss: 0.3927 - val_accuracy: 0.8315
Epoch 122/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4295 - accuracy: 0.8087 - val_loss: 0.3913 - val_accuracy: 0.8258
Epoch 123/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4298 - accuracy: 0.8115 - val_loss: 0.3943 - val_accuracy: 0.8258
Epoch 124/200
9/9 [========

9/9 [==============================] - 0s 3ms/step - loss: 0.4253 - accuracy: 0.8059 - val_loss: 0.3918 - val_accuracy: 0.8371
Epoch 176/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4259 - accuracy: 0.8101 - val_loss: 0.3885 - val_accuracy: 0.8371
Epoch 177/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4249 - accuracy: 0.8143 - val_loss: 0.3908 - val_accuracy: 0.8427
Epoch 178/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4250 - accuracy: 0.8129 - val_loss: 0.3913 - val_accuracy: 0.8371
Epoch 179/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4251 - accuracy: 0.8143 - val_loss: 0.3886 - val_accuracy: 0.8371
Epoch 180/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4250 - accuracy: 0.8087 - val_loss: 0.3922 - val_accuracy: 0.8371
Epoch 181/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4245 - accuracy: 0.8045 - val_loss: 0.3909 - val_accuracy: 0.8258
Epoch 182/200
9/9 [========